In [2]:
%load_ext autoreload
%autoreload 2

import torch
from src.architectures.feature_extractors.mobilenet import MobileNet
from src.module.multiclass import MulticlassImageClassifier
from src.architectures.head import MulticlassLinearClassificationHead
from pytorch_lightning import seed_everything

from torchinfo import summary

count_params = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)

seed_everything(42)
img = torch.rand(1, 3, 224, 224)

# summary(net, input_data=img)

Global seed set to 42


In [29]:
net = MobileNet(3, 0.3, "v3_small", True, False).cuda()

In [30]:
net.net(img.cuda()).shape

torch.Size([1, 304])

In [28]:
summary(net, input_data=img, depth=3)

Layer (type:depth-idx)                                  Output Shape              Param #
ExternalFeatureExtractor                                [1, 1280]                 --
├─Sequential: 1-1                                       --                        --
│    └─Sequential: 2-1                                  [1, 1280, 7, 7]           --
│    │    └─Conv2dNormActivation: 3-1                   [1, 16, 112, 112]         464
│    │    └─InvertedResidual: 3-2                       [1, 8, 112, 112]          320
│    │    └─InvertedResidual: 3-3                       [1, 8, 56, 56]            1,408
│    │    └─InvertedResidual: 3-4                       [1, 8, 56, 56]            1,408
│    │    └─InvertedResidual: 3-5                       [1, 16, 28, 28]           1,808
│    │    └─InvertedResidual: 3-6                       [1, 16, 28, 28]           4,352
│    │    └─InvertedResidual: 3-7                       [1, 16, 28, 28]           4,352
│    │    └─InvertedResidual: 3-8          